In [7]:
from datetime import date, datetime, timedelta
from collections import defaultdict
from scipy.optimize import minimize
from statsmodels.nonparametric.kernel_regression import KernelReg as kr 
from numpy import array
from iteration_utilities import flatten
from statsmodels.tsa.stattools import adfuller

import statsmodels.api as sm
import scipy.optimize as opt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.linalg import inv
import scipy.stats as st

def VECM(x, y, p):

    # p=p-1
    
    # First-difference the series
    dx = np.diff(x)
    dy = np.diff(y)

    Y = np.column_stack([dy, dx])
    
    beta = 1.0
    ECT = y[:-1] - beta * x[:-1]  # shape (T-1,)

    dx_lag = np.concatenate([[0], dx[:-1]])  # Δx_{t-1}
    dy_lag = np.concatenate([[0], dy[:-1]])  # Δy_{t-1}

    # Build regression matrices
    X = np.column_stack([dy_lag, dx_lag])
    
    for i in range(1, p, 1):
        
        dx_lag = np.concatenate([[0], dx_lag[:-1]])  # Δx_{t-p}
        dy_lag = np.concatenate([[0], dy_lag[:-1]])  # Δy_{t-p}
        X = np.column_stack([X, dy_lag, dx_lag])

    X = np.column_stack([ECT, X])
    
    # Step 6: Estimate coefficients using OLS: A = (X'X)^{-1} X'Y
    A = inv(X.T@X)@(X.T@Y)    
    Y_hat = X @ A
    residuals = Y - Y_hat

    T, k = X.shape
    n_eq = Y.shape[1]

    # Inverse of X'X
    XtX_inv = np.linalg.inv(X.T @ X)
    Sigma = (residuals.T @ residuals) / (T - k)
    
    # Standard errors for each coefficient in each equation
    # SE(B)
    SE = np.zeros_like(A)
    for i in range(n_eq):
        SE[:, i] = np.sqrt(np.diag(Sigma[i, i] * XtX_inv))
    
    # t-statistics
    t_stats = A / SE

    return A[:1], A[1:], t_stats[:1], t_stats[1:], residuals, X

def compute_B_matrices(A_list, gamma, beta, K):
    """
    Compute MA coefficients B_0 to B_K for a VECM(p) model:
    Δy_t = Σ A_i Δy_{t-i} + Π y_{t-1} + u_t

    Parameters:
        A_list: list of (n x n) matrices [A_1, A_2, ..., A_{p-1}]
        gamma: (n x 1) adjustment coefficient matrix
        beta: (n x 1) cointegration vector
        K: number of B_i matrices to compute

    Returns:
        B_list: list of (n x n) matrices [B_0, B_1, ..., B_K]
    """
    n = A_list[0].shape[0]
    p = len(A_list) + 1  # p is lag order in levels
    Pi = gamma @ beta.T  # (n x n) matrix

    # print(p)
    
    B_list = [np.eye(n)]  # B_0 = I

    for i in range(1, K+1):
        # Autoregressive component: sum A_j * B_{i-j}
        AR_sum = np.zeros((n, n))
        for j in range(1, min(i, p)):  # up to p-1 lags
            AR_sum += A_list[j - 1] @ B_list[i - j]

        # Cointegration component: Π * sum_{j=0}^{i-1} B_j
        CI_sum = sum(B_list[:i])
        B_i = AR_sum + Pi @ CI_sum
        B_list.append(B_i)

    return B_list

In [35]:
mt=0
n=10000
m_list=[]
p1_list=[]
p2_list=[]
sigma_u=1
c1=1
c2=0
lambda_=1
m_list.append(mt)

for i in range(1, n, 1):
    
    q1=np.random.choice([1, -1], p=[0.5, 0.5])
    q2=np.random.choice([1, -1], p=[0.5, 0.5])

    mt=mt+lambda_*q1+np.random.normal(loc=0, scale=sigma_u)
    m_list.append(mt)
    
    p1_list.append(m_list[i]+c1*q1)
    p2_list.append(m_list[i-1]+c2*q2)

In [38]:
np.var(np.diff(np.array(m_list))),\
np.corrcoef(np.diff(np.array(m_list)), np.diff(np.concatenate([[0], np.array(m_list[:-1])])))[0][1]

(2.0005118862354085, -0.010404442352217928)

In [41]:
y = np.array(p1_list) # y is the first market 
x = np.array(p2_list) # x is the second market 

p=20
K=100

IS1_lower_list=[]
IS1_upper_list=[]

IS2_lower_list=[]
IS2_upper_list=[]

gamma1_list=[]
gamma2_list=[]
gamma1_tstat_list=[]
gamma2_tstat_list=[]

Gamma, A, Gamma_tstat, A_tstat, ut, X = VECM(x, y, p)

A_list=[]
for i in range(0, p, 1):
    A_list.append(A[0+2*i:2*(i+1)])

print("Gamma 1 = {} (t stat: {}) and Gamma 2 = {} (t stat: {})".format(Gamma.T[0][0],
                                                                       Gamma_tstat.T[0][0],
                                                                       Gamma.T[1][0],
                                                                       Gamma_tstat.T[1][0]))
w1=(-1/(Gamma.T[0][0]-Gamma.T[1][0]))*Gamma.T[1][0]
w2=(1/(Gamma.T[0][0]-Gamma.T[1][0]))*Gamma.T[0][0]
f=w1*x+w2*y

print("PT/GG estimates: w 1 = {} and w 2 = {}".format(w1, w2))
print("Variance of efficient price change estimates = {}".format(np.var(np.diff(f))))
print("First-order autocorrelation of efficient price change = {}".format(
    np.corrcoef(np.diff(np.array(f)), np.diff(np.concatenate([[0], np.array(f[:-1])])))[0][1]))

gamma1_list.append(Gamma.T[0][0])
gamma2_list.append(Gamma.T[1][0])
gamma1_tstat_list.append(Gamma_tstat.T[0][0])
gamma2_tstat_list.append(Gamma_tstat.T[1][0])

beta = np.array([[1.0], [-1.0]])
# beta = np.array([[a], [-b]])

gamma = Gamma.T
A_list = A_list
B_list = compute_B_matrices(A_list, gamma, beta, K)

# print("B_1 =\n", B_list[1])
# print("B_2 =\n", B_list[2])
# print("B_3 =\n", B_list[3])

Psi = sum(B_list)
# print("Sum of B_i from i=0 to K:\n", Psi)

b=Psi[1:]

Sigma = np.cov(ut.T)
# print("Covariance matrix of ut = {}".format(Sigma))

# Cholesky: upper bound (market 1 first)
C1 = np.linalg.cholesky(Sigma)

# Cholesky: lower bound (market 2 first)
P = np.array([[0, 1], [1, 0]])
Sigma_perm = P @ Sigma @ P.T
C2_perm = np.linalg.cholesky(Sigma_perm)
C2 = P.T @ C2_perm

psi = b  # common trend loadings

# Upper bound
gamma1 = psi @ C1
IS1_upper = gamma1[0][0]**2 / np.sum(gamma1**2)
IS2_lower = 1 - IS1_upper

# Lower bound
gamma2 = psi @ C2
IS2_upper = gamma2[0][0]**2 / np.sum(gamma2**2)
IS1_lower = 1 - IS2_upper

# --- Step 4: Print results ---
print(" ")
print("IS estimates")
print(f"Market 1 IS (lower, upper): {IS1_lower:.4f} – {IS1_upper:.4f}")
print(f"Market 2 IS (lower, upper): {IS2_lower:.4f} – {IS2_upper:.4f}")

IS1_lower_list.append(IS1_lower)
IS1_upper_list.append(IS1_upper)

IS2_lower_list.append(IS2_lower)
IS2_upper_list.append(IS2_upper)

print("")
print("Structural model")
print("Variance of efficient price change estimates = {}".format(1))
print("First-order autocorrelation of efficient price change = {}".format(0))

Gamma 1 = -0.17543704953502479 (t stat: -1.3216804273123484) and Gamma 2 = 0.5961317958895899 (t stat: 22.832471052536686)
PT/GG estimates: w 1 = 0.7726229479386546 and w 2 = 0.22737705206134542
Variance of efficient price change estimates = 1.136759136304084
First-order autocorrelation of efficient price change = 0.36389025614201687
 
IS estimates
Market 1 IS (lower, upper): 0.9590 – 0.9996
Market 2 IS (lower, upper): 0.0004 – 0.0410

Structural model
Variance of efficient price change estimates = 1
First-order autocorrelation of efficient price change = 0
